In [55]:
import numpy as np
import trimesh
import open3d as o3d
import torch
import torch.nn.functional as F

import math

from shapely.geometry import LineString, Point

# np.set_printoptions(precision=4, suppress=True)
# torch.set_printoptions(precision=4, sci_mode=False)

def scale_mesh(stl_path, offset=0.1):
    mesh = trimesh.load(stl_path)    
    vertices = torch.from_numpy(mesh.vertices).float()
    vertices = vertices / vertices.abs().max() * (1.-offset)        
    return trimesh.Trimesh(vertices=vertices, faces=mesh.faces)

def get_start(mesh):
    ray_origins = np.array([[0, 0, 2],])
    ray_directions = np.array([[0, 0, -1]])

    locations, _, index_tri = mesh.ray.intersects_location(
            ray_origins=ray_origins,
            ray_directions=ray_directions)
    top_face_idx = locations[:, 2].argmax()
    start_face = index_tri[top_face_idx] 
    start_point =  locations[top_face_idx]
    start_normal = np.array([[0, 0, +1]])
    return start_face, start_point, start_normal

class Bridge:

    def __init__(self, path):
        self.path = path
        self.mesh = scale_mesh(path)
        self.faces = self.make_faces()
        self.polars = math.pi + np.array(
            np.arctan2(self.mesh.vertices[:, 1], 
                       self.mesh.vertices[:, 0]))
        self.edge_to_face = self.make_edge_to_face()

    def make_faces(self):
        faces = np.array(self.mesh.faces)
        faces.sort(axis=1)
        return faces
    
    def make_edge_to_face(self):        
        edge_to_face = {}
        for i, face in enumerate(self.faces):
            edges = self.get_face_edges(face)
            for edge in edges:
                if edge in edge_to_face:
                    edge_to_face[edge].append(i)
                else:
                    edge_to_face[edge] = [i]
        return edge_to_face
    
    def get_face_edges(self, face):
        return [(face[0], face[1]),
                (face[0], face[2]),
                (face[1], face[2]),]
        
    def get_face_ids(self, edge, past_face_ids):        
        if edge in self.edge_to_face:
            face_ids = self.edge_to_face[edge]
            
            res =  [f for f in face_ids if f not in past_face_ids]
            #print('get_face_ids', face_ids, past_face_ids, res)
            return res
        return []    
    
    def next_edges_face_id(self, edge, past_face_ids, past_edges):
        face_ids = self.get_face_ids(edge, past_face_ids)
        face_id, edges = None, []
        if face_ids:
            face_id = face_ids[0] # Pick first face             
            face =  self.faces[face_id]
            face_edges = self.get_face_edges(face)
            for face_edge in face_edges:
                if face_edge not in past_edges:
                    edges.append(face_edge)                
        return face_id, edges
            
        
    def __repr__(self):
        return f'Path: {self.path}'
        
    def scale_mesh(self, stl_path, offset=0.1):
        mesh = trimesh.load(stl_path)    
        vertices = torch.from_numpy(mesh.vertices).float()
        vertices = vertices / vertices.abs().max() * (1.-offset)        
        return trimesh.Trimesh(vertices=vertices, faces=mesh.faces)
    
def get_intersection_point(latitude, edge_vertex, edge_vertex_normals):
    ev_2d  = edge_vertex[:, :2]
    e1_pt, e2_pt = Point(ev_2d[0]),  Point(ev_2d[1])
    
    l1 = LineString(latitude)
    l2 = LineString(ev_2d)
    if l1.intersects(l2):
        pt = l1.intersection(l2)        
        xy = np.array(pt.xy)
        d1 = e1_pt.distance(pt)
        d2 = e1_pt.distance(e2_pt)
        assert d1 <= d2,  ('d1 > d2',  pt, d1, d2)
        ratio = d1 / d2 if d2 != 0 else 0.5
        xyz = ratio * edge_vertex[0] + (1-ratio) *  edge_vertex[1]
        normal =  ratio * edge_vertex_normals[0] + (1-ratio) *  edge_vertex_normals[1]
        return np.array(xyz), normal
        
    return None, None
   
def it_intersects(edge, latitude, bridge):
    edge_sel = np.array(edge)    
    edge_vertex =  bridge.mesh.vertices[edge_sel]        
    edge_vertex_normals =  bridge.mesh.vertex_normals[edge_sel]            
    return get_intersection_point(latitude, edge_vertex, edge_vertex_normals)
    

def make_latitudes(n):
    r_angle = torch.stack((
        torch.zeros(n) + 2,
        torch.arange(0, n) * (2 * math.pi / n)))
    xy = torch.stack((r_angle[0]*torch.cos(r_angle[1]),
                      r_angle[0]*torch.sin(r_angle[1])))#.t()
    latitudes = torch.stack((torch.zeros_like(xy), xy))
    return latitudes.permute(2, 0, 1).numpy()    

def get_intersection(latitude, edges, bridge):
    #print('get_intersection', edges)
    for edge in edges:
        edge_line = bridge.mesh.vertices[np.array(edge)]                
        point, normal = it_intersects(edge, latitude, bridge)
        if point is not None:            
            return point, normal,  edge    
    return None, None, None
    
def gather_path(latitude, face_id, point, normal, bridge):    
    face = bridge.faces[face_id]
    edges = bridge.get_face_edges(face)
    path = { 'points': [point], 'normals': [normal]}    
    point, normal, edge = get_intersection(latitude, edges, bridge)        
    path['points'].append(point) 
    path['normals'].append(normal) 
    past_face_ids, past_edges = [face_id], edges.copy()
    while True:        
        face_id, edges = bridge.next_edges_face_id(edge, past_face_ids, past_edges)
        #print(past_edges)
        if face_id is not None:
            past_face_ids.append(face_id)
            past_edges = past_edges + edges
            point, normal, edge = get_intersection(latitude, edges, bridge)
            if point is not None:                
                path['points'].append(point) 
                path['normals'].append(normal) 
            else: 
                break
        else:
            break
    path['points'] =  np.array(path['points'])
    path['normals'] =  np.array(path['normals'])
    return path

def gather_all_paths(stl_file, latitudes_num):
    latitudes = make_latitudes(latitudes_num)
    bridge = Bridge(stl_file)  
    face_id, point, normal = get_start(bridge.mesh)
    paths = []
    for latitude in latitudes:
        #print(latitude)
        path = gather_path(latitude, face_id, point, normal, bridge)
        paths.append(path)
    return paths

def cumulative_distances(path):
    cumulative = [0]
    for i in range(0, len(path)-1):        
        dist = np.linalg.norm(path[i] - path[i+1])
        cumulative.append(dist+cumulative[-1])
    cumulative = np.array(cumulative)
    return cumulative

def get_sample(pos, normed, path_points, path_normals):
    i = 1
    while i < len(normed):
        d0, d1 = normed[i-1], normed[i]
        if pos <= d1:
            ratio = (pos - d0) / (d1 - d0) 
            pnt = ratio * path_points[i-1] + (1 - ratio) * path_points[i]
            nrm = ratio * path_normals[i-1] + (1 - ratio) * path_normals[i]
            return pnt, nrm, i
        i += 1
    raise (pos, normed, path)
    
def sample_path(path, samples_num):
    cumulative = cumulative_distances(path['points'])
    normed =  cumulative / cumulative[-1]
    step = 1. / samples_num
    position = np.arange(0, 1, step) + step
    samples = {'points': [], 'normals': []}
    start = 0
    for pos in position:
        pnt, nrm, start = get_sample(pos, normed[start:], path['points'][start:],
                               path['normals'][start:])
        samples['points'].append(pnt)
        samples['normals'].append(nrm)
        
    samples['points'] =  np.array(samples['points'])
    samples['normals'] =  np.array(samples['normals'])   
    return samples

def sample_all_paths(paths, samples_num):
    all_samples = []
    for path in paths:
        path_samples =  sample_path(path, samples_num)        
        all_samples.append(path_samples)
    res_points = np.array([p['points'] for p in all_samples])
    res_normals = np.array([p['normals'] for p in all_samples])

    assert res_points.shape ==  res_normals.shape
    n, h = len(paths) // 2, len(res_points) // 2
    # # Concatenate opposite angles 0-180, 10:190, ...170:350
    return (
        np.concatenate((np.flip(res_points[h:], axis=1), res_points[:h]), axis=1),
        np.concatenate((np.flip(res_normals[h:], axis=1), res_normals[:h]), axis=1)
    )

stl_dir = "./data/stls/"

file_name = "./data/flame_sample.stl"
print(file_name)
latitudes_num = 12
paths = gather_all_paths(file_name, latitudes_num)
print(len(paths))
paths

./data/flame_sample.stl
12


[{'points': array([[ 0.00000000e+00,  0.00000000e+00,  7.75498016e-01],
         [ 4.11246530e-02, -1.84375346e-02,  7.71572922e-01],
         [ 5.04736177e-02, -2.89626983e-02,  7.69472404e-01],
         [ 1.19095400e-01, -4.05230266e-02,  7.49638117e-01],
         [ 1.53009226e-01,  8.45968723e-04,  7.37152952e-01],
         [ 1.88770409e-01,  4.71577798e-02,  7.18208320e-01],
         [ 2.39924471e-01,  3.69903264e-02,  6.93816864e-01],
         [ 2.46040177e-01,  2.79501742e-02,  6.91834382e-01],
         [ 2.80009722e-01,  2.00535906e-02,  6.70738512e-01],
         [ 2.94600086e-01,  1.29970741e-02,  6.62037834e-01],
         [ 3.17646782e-01,  7.92418420e-03,  6.43859992e-01],
         [ 3.34676375e-01,  2.85129808e-03,  6.30341692e-01],
         [ 3.51613840e-01, -1.09428354e-03,  6.14287081e-01],
         [ 3.69368049e-01, -4.79597598e-03,  5.97157919e-01],
         [ 3.83159103e-01, -7.59798661e-03,  5.79769922e-01],
         [ 4.01009196e-01, -9.76589508e-03,  5.56284871e-01]

In [57]:
samples_num = 50
sample_points, sample_normals = sample_all_paths(paths, samples_num)
#print(samples.shape)
sample_points.shape, sample_normals.shape

((6, 100, 3), (6, 100, 3))

In [6]:
file_name = "./data/flame_sample.stl"
mesh = trimesh.load(file_name)    
mesh

<trimesh.Trimesh(vertices.shape=(5022, 3), faces.shape=(10004, 3))>

In [7]:
mesh.fix_normals()
mesh.vertex_normals

array([[-0.15443365, -0.53592453, -0.83002117],
       [ 0.34562465,  0.84241906, -0.41336875],
       [ 0.93960923,  0.3027177 , -0.1596762 ],
       ...,
       [-0.57373961, -0.81717091,  0.05526809],
       [ 0.56690512, -0.69829826, -0.43703333],
       [-0.95350546, -0.24356434,  0.17749295]])

In [6]:
vertex_normals = mesh.vertex_normals

(vertex_normals * vertex_normals).sum(axis=-1)

array([1., 1., 1., ..., 1., 1., 1.])

In [7]:
mesh.vertices.shape, mesh.vertex_normals.shape

((5022, 3), (5022, 3))

In [8]:
mesh.vertices[np.array([4778, 4994])]

TrackedArray([[ 0.06926834,  0.02851255,  0.03206284],
              [ 0.08353534,  0.01654455, -0.05442716]])

In [9]:
mesh.vertex_normals[np.array([4778, 4994])]

array([[ 0.5146398 , -0.15328347,  0.84359354],
       [ 0.75374666,  0.63112276, -0.18316667]])

In [9]:
stl_dir = "./data/stls/"

file_name = "./data/flame_sample.stl"
print(file_name)
latitudes_num = 12
paths = gather_all_paths(file_name, latitudes_num)
print(len(paths))

# samples_num = 500
# samples = sample_all_paths(paths, samples_num)
# print(samples.shape)

./data/flame_sample.stl
12


In [19]:
paths

[array([[array([0.        , 0.        , 0.77549802]), array([[0, 0, 1]])],
        [array([ 0.04112465, -0.01843753,  0.77157292]),
         array([ 0.18145442, -0.0453527 ,  0.97760244])],
        [array([ 0.05047362, -0.0289627 ,  0.7694724 ]),
         array([ 0.19256774, -0.08718079,  0.97442069])],
        [array([ 0.1190954 , -0.04052303,  0.74963812]),
         array([ 0.29699137, -0.11044189,  0.94660803])],
        [array([0.15300923, 0.00084597, 0.73715295]),
         array([0.35039987, 0.05513898, 0.93384447])],
        [array([0.18877041, 0.04715778, 0.71820832]),
         array([0.40277055, 0.13861283, 0.90431647])],
        [array([0.23992447, 0.03699033, 0.69381686]),
         array([0.48656172, 0.13837617, 0.86137621])],
        [array([0.24604018, 0.02795017, 0.69183438]),
         array([0.49469928, 0.12168242, 0.85919716])],
        [array([0.28000972, 0.02005359, 0.67073851]),
         array([0.56002909, 0.11588925, 0.81697881])],
        [array([0.29460009, 0.01299

In [37]:
samples['normals']

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [52]:
all_samples = []
for path in paths:
    path_samples =  sample_path(path, samples_num)
    #return path_samples
    all_samples.append(path_samples)
all_samples    
res_points = np.array([p['points'] for p in all_samples])
res_normals = np.array([p['normals'] for p in all_samples])

assert res_points.shape ==  res_normals.shape
n = len(paths) // 2
# res = np.array(all_samples)    
h = len(res_points) // 2
# # Concatenate opposite angles 0-180, 10:190, ...170:350

np.concatenate((np.flip(res_points[h:], axis=1), res_points[:h]), axis=1)
np.concatenate((np.flip(res_normals[h:], axis=1), res_normals[:h]), axis=1)

array([[[-0.94063263,  0.17715854,  0.28800033],
        [-0.92746032,  0.15590909,  0.33286021],
        [-0.88878568,  0.10005258,  0.43445145],
        ...,
        [ 0.97876201, -0.10457814,  0.02555419],
        [ 0.77650942, -0.02607837,  0.61478923],
        [ 0.77583336, -0.02588068,  0.61566508]],

       [[-0.63654341, -0.16024686,  0.74922544],
        [-0.54675213, -0.23517299,  0.79511151],
        [-0.71659072, -0.07593269,  0.6839006 ],
        ...,
        [ 0.71291654, -0.24521864,  0.65001952],
        [ 0.81263784, -0.14822871,  0.55576787],
        [ 0.68772473, -0.26972044,  0.67382957]],

       [[ 0.32893701, -0.44547097,  0.83249978],
        [ 0.16073613, -0.44594566,  0.85399168],
        [-0.00746474, -0.44642034,  0.87548358],
        ...,
        [ 0.71297705,  0.01082977,  0.68547096],
        [ 0.59394797,  0.0611128 ,  0.80084936],
        [ 0.63711232,  0.06544196,  0.76657328]],

       [[ 0.58790283, -0.29971488,  0.75014956],
        [ 0.62420814, -0

In [50]:
sample_points = np.array([p['points'] for p in all_samples])
sample_points.shape

(12, 50, 3)

In [49]:
for p in paths:
    print( p['points'] .shape)

(195, 3)
(102, 3)
(75, 3)
(171, 3)
(56, 3)
(59, 3)
(60, 3)
(62, 3)
(66, 3)
(73, 3)
(78, 3)
(159, 3)


In [45]:
paths[0]['points'].shape

(195, 3)